# Kaggle Titanic Competition - Model Building, Testing, and Selection
- Problem Description: The sinking of the Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew. While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others. In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc). 
- Author: Kimberly Gaddie
- Date Last Updated: 17 May 2021

#### Import Libraries and Datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (StandardScaler, MinMaxScaler, 
                                   LabelEncoder, OneHotEncoder)

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier,
                              VotingClassifier)

from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict,
                                     StratifiedKFold, learning_curve, train_test_split)

from sklearn.metrics import (confusion_matrix, roc_curve, auc, roc_auc_score, 
                             accuracy_score, f1_score, explained_variance_score)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from yellowbrick.model_selection import FeatureImportances
import xgboost as xgb

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
"notebook.output.textLineLimit": 500

df_full = read.csv('df_full_featured.csv')
df_train = read.csv('train.csv')
df_test = read.csv('test.csv')

## Model Building
- Unpack the full dataset back to test & train
- Feature Selection
- HyperParameter Tuning

In [ ]:
df_full.columns

In [ ]:
known = df_full[df_full['df_type'] == 'Train']
val = df_full[df_full['df_type'] == 'Test']
print(f'{len(known)} Training Records & {len(val)} Validation Records')

In [ ]:
random_state = 5254 # Reproducability!!!

# Usually use 75/25 or 80/20 split... But w/ low n... Let's do 85/15
    # This will probably bias our test metrics lower 'indicating' a 'worse'
    # performing model (less n, higher probability of outlier / fringe case
    # scenarios not learned by model...)
train, test = train_test_split(known, test_size=0.15, random_state=random_state)

print(f'''{len(train)} Training Records & {len(test)} Testing Records.
      Split of {len(train) / len(known)} | {len(test) / len(known)}''')

In [ ]:
# Let's start with all of these... We should eventually pare the list down

features = ['female', 'male', 'Fare', 'New Fare', 'Pclass', 'is_minor', 
            'is_group', 'Total Family', 'Upper', 'Miss', 'Mrs', 'fare_divider', 
            'Embarked_S', 'Embarked_Q', 'Embarked_C', 'cabin_B', 'cabin_C', 'cabin_D',
            'cabin_G', 'cabin_F']

In [ ]:
## Separate train features and label 
Y_train = train["Survived"].astype(int)
X_train = train[features]

y_test = test['Survived']
test = test[features]

# Cross validate model with Kfold stratified cross val
folds = StratifiedKFold(n_splits=5)

### Test 'Out of the Box' solutions
- We can narrow down model types by Identifying the better performing models
- This doesn't include any hyperparameter tuning!!! 

In [ ]:
model_types = []
cv_results = []
cv_means = []
cv_std = []

model_types.append(KNeighborsClassifier())
model_types.append(DecisionTreeClassifier(random_state=random_state))
model_types.append(RandomForestClassifier(random_state=random_state))
model_types.append(SVC(random_state=random_state))
model_types.append(GradientBoostingClassifier(random_state=random_state))

In [ ]:
model_types

In [ ]:
for model in model_types:
    cv_results.append(cross_val_score(model, X_train, Y_train, scoring = 'accuracy', cv=folds, n_jobs=4))
    
for result in cv_results:
    cv_means.append(np.median(result))
    cv_std.append(np.std(result))

In [ ]:
model_performance = pd.DataFrame({
    'Cross_Val_Median': cv_means,
    'Cross_Val_Errors': cv_std,
    'Model_Types': ['KNeighbors',
                   'DecisionTree',
                   'RandomForest',
                   'SVC',
                   'GradientBoost']
})

In [ ]:
plotted = sns.barplot('Cross_Val_Median', 'Model_Types', data=model_performance, orient='h', **{'xerr': cv_std})
plotted.set_xlabel('Median Model Accuracy')

#### HyperParameter Tuning
- Random Forest (I just like it, but might have better luck w/ Decision Tree
    or Gradient Boosting)
- GB (most likely will switch to xgboost package here, I like the 
    implementation / api better than in sklearn)
- Decision Tree
- KNN (MAYBE -- I am super unconvinced it will actually provide meaningful
    value)
- Can we ensemble these together..? They may provide improved performance in a
    grouped competitive voting situation...

##### Random Forest
- We'll use a grid search cross validation method... Not the fastest, cpu 
    intensive, but easiest heuristic to implement. 
- We should consider other methods - Optuna, Baysian, etc....

In [ ]:
# Initialize decision tree
# Not well defined in documentation, let's try both
bootstrap = [True, False]

# Number of trees in forest... Must be careful!
    # Exceeding observations = over-fitting and possible 'perfect' predition
n_estimators = [35, 40, 45, 50, 100, 200, 300]

# Maximum depth of the tree (could use None (default), but overfitting...)
max_depth = [5, 8, 10, 13, 15, 18, 20, 25, 30]

# Threshold for creating a new split
min_samples_split = [2, 5, 10, 15, 100]

# Min samples required to keep a leaf
min_samples_leaf = [1, 2, 5, 10]

# Function used to measure quality of a split
criterion=['gini','entropy']


forest = RandomForestClassifier(random_state = random_state) 

hyperF = dict(bootstrap = bootstrap,
              n_estimators = n_estimators,
              max_depth = max_depth,  
              min_samples_split = min_samples_split, 
              min_samples_leaf = min_samples_leaf,
              criterion = criterion)

gridF = GridSearchCV(forest, hyperF, cv = folds, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, Y_train)

# What did we determine the best estimator was?
# We can pickle this for later?
# If we use MLFlow this can be an artifact
print(bestF.best_params_)

# What did we determine the best parameters were?
# We can pickle this for later?
# If we use MLFlow this can be an artifact
print(bestF.best_estimator_)

y_predF = bestF.predict(test)


In [ ]:
# How did our new model do?
# AUC - Area Under the Curve
# F1 Score - Calculated from percision and recall
    ## Precision = correctly predicted positives / total predicted positives
    ## Recall = correctly predicted positives / total true positives
# Accuracy - (TP + TN)/(TP + TN + FP + FN)

y_scores = bestF.predict_proba(test)[:,1]
fpr, tpr, thres = roc_curve(y_test,  y_scores)

print('AUC: ', auc(fpr, tpr))

print('F1: ',f1_score(y_test, y_predF))

print('Accuracy: ',accuracy_score(y_test, y_predF))

In [ ]:
# Can we visualize this? 

# compute true positive rate and false positive rate
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_scores)

# Define function to plot them
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

# Set figure size    
plt.figure(figsize=(14, 7))

# Run function
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()


##### Decision Tree
- We'll use a grid search cross validation method... Not the fastest, cpu 
    intensive, but easiest heuristic to implement. 
- We should consider other methods - Optuna, Baysian, etc....

In [ ]:
# If we don't want to run this in isolation, we could steal these from
    # above

# Function used to measure quality of a split
criterion=['gini','entropy']

# Maximum depth of the tree (could use None (default), but overfitting...)
max_depth = [5, 8, 10, 13, 15, 18, 20, 25, 30]

# Threshold for creating a new split
min_samples_split = [2, 5, 10, 15, 100]

# Min samples required to keep a leaf
min_samples_leaf = [1, 2, 5, 10]

param_grid = {'criterion': criterion,
              'max_depth': max_depth,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

tree = DecisionTreeClassifier(random_state=random_state)
gs = GridSearchCV(tree,param_grid,cv=folds)
gs.fit(X_train,Y_train)

best = gs.best_estimator_
print(best)

# Predict the response for test dataset
y_pred2 = best.predict(test)

In [ ]:
y_pred_proba = best.predict_proba(test)[:,1]
fpr, tpr, thres = roc_curve(y_test,  y_pred_proba)

print('AUC: ', auc(fpr, tpr))

print('F1: ',f1_score(y_test, y_pred2))

print('Accuracy: ',accuracy_score(y_test, y_pred2))

In [ ]:
# Create a new matplotlib figure
fig = plt.figure()
ax = fig.add_subplot()

fts = FeatureImportances(best, ax=ax)
fts.fit(X_train, Y_train)
fts.poof()

#### XGBoost
-Switch to xgboost package, as noted earlier

In [ ]:
# #Convert into data structure for xgb
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)

In [ ]:
n_estimators = [5, 10, 20, 50, 100, 200, 300]
max_depth = [3, 6, 9, 12]
min_child_weight = [1, 5, 10]
gamma = [0, 1, 2]
subsample = [0.5, 0.75, 1]
colsample_bytree = [0.5, 1]
reg_alpha = [0.001, 0.01, 0.1]
reg_lambda = [0.001, 0.01, 0.1]
scale_pos_weight = [1.0]

In [ ]:
param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'reg_lambda': reg_lambda,
              'reg_alpha': reg_alpha,
              'scale_pos_weight': scale_pos_weight}

In [ ]:
gsearch = GridSearchCV(
        estimator=xgb.XGBClassifier(
            objective = "binary:logistic",
            nthread = 1,
            seed = random_state,
            verbosity = 1,
            metric = 'rmse'
        ),
        param_grid=param_grid,
        error_score=0,
        n_jobs=32,
        verbose=1,
        return_train_score=True,
        iid=False,
        cv=folds
    )

In [ ]:
gsearch.fit(X_train, Y_train)

print(f'Mean test scores of Grid search: {gsearch.cv_results_["mean_test_score"].mean()}')
print(f'Selected hyper parameters from grid search: {gsearch.best_params_}')

In [ ]:
# Let's see how we did
y_pred3 = gsearch.predict(test)
y_pred3_score = gsearch.predict_proba(test)[:,1]

In [ ]:
fpr, tpr, thres = roc_curve(y_test,  y_pred3_score)

print('AUC: ', auc(fpr, tpr))

print('F1: ',f1_score(y_test, y_pred3))

print('Accuracy: ',accuracy_score(y_test, y_pred3))

In [ ]:
gsearch.best_score_
xgb_best = gsearch.best_estimator_
print(xgb_best)

xgb.plot_importance(xgb_best)